### Importacion de librerias

In [4]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")
import googlemaps
from datetime import time


### Cargar datos

In [ ]:
data=pd.read_csv('Accidentalidad_en_Barranquilla_20231015.csv')

In [ ]:

data

In [ ]:
data = data.dropna(subset=['SITIO_EXACTO_ACCIDENTE'])


### PASO 1: Geocodificación

Podemos observar que las direcciones deben codificarse de manera que sea mucho más facil localizar el accidente. Procederemos de la siguiente manera:
1. Crear un funcion que nos permita 'geocodificar' las direcciones en latitud y longitud, como dos nuevas columnas del dataset.
2. Dados los costos de usar la api de googlemaps aplicaremos la funcion solo a las direcciones UNICAS. 
3. Finalmente exportaremos nuestros resultados en un archivo csv. 

In [ ]:

# Inicializa el cliente de Google Maps
gmaps = googlemaps.Client(key='xxxxxxxxxxxxxxxxxxxxx')

# Define una función para obtener la latitud y longitud de una dirección
def obtener_latitud_longitud(direccion, gmaps_client):
    try:
        # Utiliza el geocodificador de lote para reducir las llamadas a la API
        geocode_result = gmaps_client.geocode(direccion + ', Barranquilla, Colombia')

        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            latitud = location['lat']
            longitud = location['lng']
            return latitud, longitud
        else:
            return None, None
    except Exception as e:
        print(f"Error en la geocodificación: {str(e)}")
        return None, None




In [ ]:
#mascara_duplicados = data['SITIO_EXACTO_ACCIDENTE'].duplicated(keep='first')


In [ ]:
#data_unicos=data[~mascara_duplicados]

In [ ]:
#data_unicos.head()

In [ ]:
#Apliación de la función
#data_unicos['latitud'], data_unicos['longitud'] = zip(*data_unicos['SITIO_EXACTO_ACCIDENTE'].apply(lambda x: obtener_latitud_longitud(x, gmaps)))

In [ ]:
#data_unicos.head()

In [ ]:
#data_unicos.to_csv('datos_unicos.csv', index=False)  guardar dataset con las direcciones codificadas en latitud y longitud 


### PASO 2: Manipulación de datos 

In [ ]:
data_unicos=pd.read_csv('datos_unicos.csv')

In [ ]:
data_unicos.isna().sum()

la geocodificacion dada por nuestra funcion devolvió algunos valores Nulos en las columnas latitud y longitud(limites de la api)  , para efectos practicos eliminemos esos valores.


In [ ]:
data_unicos=data_unicos.dropna(subset=['latitud','longitud'], axis=0)

Procederemos de la siguiente forma:
1. Tomaremos las direcciones duplicadas y las uniremos con las UNICAS.
2. Se observará que para los valores duplicados, sus respectivas columnas 'latitud','longitud' serán nulas. 
3. Agrupando por SITIO_EXACTO_ACCIDENTE y utilizando una funcion lambda, podremos finalmente salvar la mayoria de registros asignandoles sus respectivos valores.

In [ ]:
#1
mascara_duplicados = data['SITIO_EXACTO_ACCIDENTE'].duplicated(keep='first')


In [ ]:
df=data[mascara_duplicados]

In [ ]:
#2
df_merged = pd.merge(data_unicos, df, how='outer')


In [ ]:
df_merged.tail()

In [ ]:
#3
grouped = df_merged.groupby('SITIO_EXACTO_ACCIDENTE')
df_merged['latitud'] = grouped['latitud'].transform(lambda x: x.fillna(x.max()))
df_merged['longitud'] = grouped['longitud'].transform(lambda x: x.fillna(x.max()))


In [ ]:
df_merged.isna().sum()

Como se observa persisten algunos valores NA en latitud y longitud que no puedieron ser geocodificados. Como son una cantidad pequeña podemos corregir:

In [ ]:
# Primero, consideremos una máscara booleana para identificar las filas con valores faltantes en 'latitud' y 'longitud'
masc = df_merged['latitud'].isna()
masc1 = df_merged['longitud'].isna()
# Luego, aplica la función solo a las filas que cumplen con la máscara y actualiza 'latitud' y 'longitud' en esas filas
df_merged.loc[masc, 'latitud'] = df_merged.loc[masc, 'SITIO_EXACTO_ACCIDENTE'].apply(lambda x: obtener_latitud_longitud(x, gmaps)[0])
df_merged.loc[masc1, 'longitud'] = df_merged.loc[masc1, 'SITIO_EXACTO_ACCIDENTE'].apply(lambda x: obtener_latitud_longitud(x, gmaps)[1])


In [ ]:
df_merged.isna().sum()

In [ ]:
df_merged.to_csv('datos.csv', index=False)

## DESDE AQUI 

In [5]:
datos=pd.read_csv('datos.csv')

In [6]:
datos

,FECHA_ACCIDENTE,AÑO_ACCIDENTE,MES_ACCIDENTE,DIA_ACCIDENTE,HORA_ACCIDENTE,GRAVEDAD_ACCIDENTE,CLASE_ACCIDENTE,SITIO_EXACTO_ACCIDENTE,CANT_HERIDOS_EN _SITIO_ACCIDENTE,CANT_MUERTOS_EN _SITIO_ACCIDENTE,CANTIDAD_ACCIDENTES,latitud,longitud
0,2015-01-01T00:00:00.000,2015,enero,Jue,03:30:00:PM,Con heridos,Choque,CL 45 CR 19,2.0,NaN,1,10.958364,-74.794731
1,2015-01-01T00:00:00.000,2015,enero,Jue,04:20:00:AM,Solo daños,Choque,CRA 15 CLLE 21,NaN,NaN,1,10.953501,-74.776203
2,2015-01-01T00:00:00.000,2015,enero,Jue,04:40:00:PM,Con heridos,Choque,CRA 14 CLLE 35,2.0,NaN,1,10.951457,-74.788801
3,2015-01-01T00:00:00.000,2015,enero,Jue,04:50:00:PM,Con heridos,Atropello,CRA 6 CLLE 90,1.0,NaN,1,10.945047,-74.823964
4,2015-01-01T00:00:00.000,2015,enero,Jue,06:00:00:AM,Solo daños,Choque,CRA 6 CLLE 92,NaN,NaN,1,10.944513,-74.825546
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40891,31/05/2023,2023,Mayo,Miercoles,09:40:00:am,Con heridos,Choque,CALLE 100 CARRERA 37,1.0,NaN,1,10.981969,-74.832339
40892,31/05/2023,2023,Mayo,Miercoles,10:50:00:am,Con heridos,Atropello,CRA 6 CALLE 110,1.0,NaN,1,11.033264,-74.830955
40893,31/07/2023,2023,Julio,Lunes,07:40:00:am,Con heridos,Atropello,CARRERA 44 CON CALLE 75,1.0,NaN,1,10.994334,-74.809761
40894,31/08/2023,2023,Agosto,Jueves,02:30:00:am,Con heridos,Choque,CALLE 4 CARRERA 38,2.0,NaN,1,10.971208,-74.764606


In [7]:
# Notese que en la columna FECHA_ACCIDENTE hay distintos tipos de formatos. Procedamos a uniformarlos

def convertir_a_formato_uniforme(fecha):
    try:
        return pd.to_datetime(fecha)
    except:
        return None

# Aplica la función a la columna 'FECHA_ACCIDENTE'
datos['FECHA_ACCIDENTE'] = datos['FECHA_ACCIDENTE'].apply(convertir_a_formato_uniforme)

Eliminemos las columnas AÑO_ACCIDENTE	MES_ACCIDENTE	DIA_ACCIDENTE y agreguemos HORA_ACCIDENTE(convertir a formato datetime) a la fecha.

In [ ]:
datos['HORA_ACCIDENTE']

Notese que la hora está dada en un formato que no reconoce pandas. Así, mediante la siguiente función lo cambiaremos

In [8]:
def convertir_formato(time_str):
    # Separa la cadena en partes usando ":"
    parts = time_str.split(':')
    # Extrae las horas, minutos y segundos
    hour = int(parts[0])
    minute = int(parts[1])
    second = int(parts[2])
    # Verifica si es PM o pm y ajusta las horas
    if (parts[3] == 'PM' or parts[3] == 'pm'):
        if hour != 12:
            hour += 12
    elif hour == 12:
        # Si es AM y la hora es 12, cámbiala a 0
        hour = 0
    return time(hour, minute, second)

# Aplica la función a la columna 'HORA_ACCIDENTE'
datos['HORA_ACCIDENTE'] = datos['HORA_ACCIDENTE'].apply(convertir_formato)

In [9]:
#Cambiar a formato datetime hora
datos['HORA_ACCIDENTE'] = pd.to_datetime(datos['HORA_ACCIDENTE'], format='%H:%M:%S').dt.time


In [10]:
#Unir las columnas HORA_ACCIDENTE y FECHA_ACCIDENTE
datos['FECHA_ACCIDENTE'] =datos['FECHA_ACCIDENTE'] + pd.to_timedelta(datos['HORA_ACCIDENTE'].astype(str))


In [11]:
# Removiendo columnas innecesarias: AÑO_ACCIDENTE, MES_ACCIDENTE, DIA_ACCIDENTE, HORA_ACCIDENTE
datos=datos.drop(columns=['AÑO_ACCIDENTE',
 'MES_ACCIDENTE',
 'DIA_ACCIDENTE',
 'HORA_ACCIDENTE','SITIO_EXACTO_ACCIDENTE'])

In [27]:
datos['CANT_HERIDOS_EN _SITIO_ACCIDENTE'].isna().sum()

0

In [26]:
datos['CANT_HERIDOS_EN _SITIO_ACCIDENTE']=datos['CANT_HERIDOS_EN _SITIO_ACCIDENTE'].fillna(0)

In [29]:
datos['CANT_HERIDOS_EN _SITIO_ACCIDENTE']=datos['CANT_HERIDOS_EN _SITIO_ACCIDENTE'].astype(int)

In [30]:
datos

,FECHA_ACCIDENTE,GRAVEDAD_ACCIDENTE,CLASE_ACCIDENTE,CANT_HERIDOS_EN _SITIO_ACCIDENTE,CANT_MUERTOS_EN _SITIO_ACCIDENTE,CANTIDAD_ACCIDENTES,latitud,longitud
0,2015-01-01 15:30:00,Con heridos,Choque,2,NaN,1,10.958364,-74.794731
1,2015-01-01 04:20:00,Solo daños,Choque,0,NaN,1,10.953501,-74.776203
2,2015-01-01 16:40:00,Con heridos,Choque,2,NaN,1,10.951457,-74.788801
3,2015-01-01 16:50:00,Con heridos,Atropello,1,NaN,1,10.945047,-74.823964
4,2015-01-01 06:00:00,Solo daños,Choque,0,NaN,1,10.944513,-74.825546
...,...,...,...,...,...,...,...,...
40891,2023-05-31 09:40:00,Con heridos,Choque,1,NaN,1,10.981969,-74.832339
40892,2023-05-31 10:50:00,Con heridos,Atropello,1,NaN,1,11.033264,-74.830955
40893,2023-07-31 07:40:00,Con heridos,Atropello,1,NaN,1,10.994334,-74.809761
40894,2023-08-31 02:30:00,Con heridos,Choque,2,NaN,1,10.971208,-74.764606


In [38]:
datos['CANT_MUERTOS_EN _SITIO_ACCIDENTE']=datos['CANT_MUERTOS_EN _SITIO_ACCIDENTE'].astype(int).fillna(0)

In [47]:
datos

,FECHA_ACCIDENTE,GRAVEDAD_ACCIDENTE,CLASE_ACCIDENTE,CANT_HERIDOS_EN _SITIO_ACCIDENTE,CANT_MUERTOS_EN _SITIO_ACCIDENTE,CANTIDAD_ACCIDENTES,latitud,longitud
0,2015-01-01 15:30:00,Con heridos,Choque,2,0,1,10.958364,-74.794731
1,2015-01-01 04:20:00,Solo daños,Choque,0,0,1,10.953501,-74.776203
2,2015-01-01 16:40:00,Con heridos,Choque,2,0,1,10.951457,-74.788801
3,2015-01-01 16:50:00,Con heridos,Atropello,1,0,1,10.945047,-74.823964
4,2015-01-01 06:00:00,Solo daños,Choque,0,0,1,10.944513,-74.825546
...,...,...,...,...,...,...,...,...
40891,2023-05-31 09:40:00,Con heridos,Choque,1,0,1,10.981969,-74.832339
40892,2023-05-31 10:50:00,Con heridos,Atropello,1,0,1,11.033264,-74.830955
40893,2023-07-31 07:40:00,Con heridos,Atropello,1,0,1,10.994334,-74.809761
40894,2023-08-31 02:30:00,Con heridos,Choque,2,0,1,10.971208,-74.764606


In [45]:
#Exportando dataset listo para analisis
datos.to_csv('data.csv', index=False)